# Initialize imports

In [3]:
from classes import *
from functions import *
import pandas as pd
import numpy as np
from DLF import DLF
from FDLF import FDLF

# Setting up Y_Bus, bus and line objects for all task

In [6]:
line_data = ReadCsvFile('./files/network_configuration_line_data.csv')
bus_data = ReadCsvFile('./files/network_configuration_bus_data.csv')

Sbase = 100 # MVA
Ubase = 230 # kV
num_buses = len(bus_data)
YBus = BuildYbusMatrix(line_data, num_buses)
bus_overview = setupBusType(bus_data)
BusList = buildBusList(bus_data, Sbase, bus_overview)

df_YBus = pd.DataFrame(YBus)
display(df_YBus)

,0,1,2,3,4
0,6.250-18.695j,-5.000000+15.000000j,-1.250000+ 3.750000j,0.000000+ 0.000000j,0.00+ 0.00j
1,-5.000+15.000j,10.833333-32.415000j,-1.666667+ 5.000000j,-1.666667+ 5.000000j,-2.50+ 7.50j
2,-1.250+ 3.750j,-1.666667+ 5.000000j,12.916667-38.695000j,-10.000000+30.000000j,0.00+ 0.00j
3,0.000+ 0.000j,-1.666667+ 5.000000j,-10.000000+30.000000j,12.916667-38.695000j,-1.25+ 3.75j
4,0.000+ 0.000j,-2.500000+ 7.500000j,0.000000+ 0.000000j,-1.250000+ 3.750000j,3.75-11.21j


# Newton Raphson Load Flow Algorithm

In [3]:
P_spec, Q_spec = findKnowns(bus_data, Sbase)
v_guess, dirac_guess = findUnknowns(bus_overview, bus_data)

jacobian_matrix = buildJacobian(BusList, P_spec, Q_spec, v_guess, dirac_guess, YBus)

deltaP = calcP(BusList, P_spec, YBus, Sbase)
deltaQ = calcQ(BusList, Q_spec, YBus, Sbase)
knowns = np.concatenate((deltaP, deltaQ), axis= 0)

# df_inv_jac = pd.DataFrame(np.linalg.pinv(df_jac))
# print(df_inv_jac)
unknowns = calcDeltaUnknowns(jacobian_matrix, knowns)
print(unknowns)

updateVoltageAndAngleList(unknowns, dirac_guess, v_guess)
updateBusList(BusList, dirac_guess, v_guess)




[[ 0.0314373 ]
 [ 0.00898817]
 [-0.0162399 ]
 [-0.02418557]
 [-0.04669507]
 [-0.02927996]
 [-0.0090333 ]]


## Output format
X<br>Bus no.|Bus Information<br>Volts (pu)|<br>Angle (deg)|Generation<br>(MW)|Generation<br>(MVAr)|Load<br>(MW)|Load<br>(MVAr)|<br>Bus type|<br>To Bus no.|Line Flow<br>(MW)|Line Flow<br>(MVAr)|
---|---|---|---|---|---|---|---|---|---|---
|1|1.06|0|-|-|0|0|SLACK|2<br>3|||
|2|||40|-|20|10|PV|1<br>3<br>4<br>5|||
|3|||0|0|45|15|PQ|1<br>2<br>4|||
|4|||0|0|40|5|PQ|2<br>3<br>5|||
|5|||0|0|60|10|PQ|2<br>4|||


# Decoupled Load Flow

In [ ]:
dlf_jacobian = calcDecoupledJacobian(BusList, P_spec, Q_spec, v_guess, dirac_guess, YBus)
print(dlf_jacobian)
dlf_unknowns= calcDecoupledDiracVoltage(dlf_jacobian, knowns)
print(dlf_unknowns)

# Fast Decoupled Load Flow

In [2]:
FDLF()


5


# DC Power Flow